<a href="https://colab.research.google.com/github/ueki5/colaboratory/blob/main/playground-020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torchviz | tail -n 1
!pip install torchinfo | tail -n 1
!pip install japanize-matplotlib | tail -n 1
!pip install plotly | tail -n 1

In [2]:
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import japanize_matplotlib
japanize_matplotlib.japanize()
from tqdm.notebook import tqdm
import torch
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torchviz import make_dot

# 各種定数
batch_size = 500 # ミニバッチのサイズ指定
num_epochs = 10 # 繰り返し回数
lr = 0.01  # 学習率
dim_hidden = 128 # 隠れ層の次元数
dim_output = 10 # 出力層の次元数
num_layer = 2 # レイヤー数
use_cuda = True # CUDA使用

# # デバイスの割り当て
# if use_cuda:
#   device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# else:
#   device = torch.device("cpu")
# print(device)

# ダウンロード先ディレクトリ名
data_root = './data'

# テンソル化 ＋ 正規化 ＋ １階テンソル化
transform = transforms.Compose([
  # データのテンソル化
  transforms.ToTensor(),
  # データの正規化(Normalize(μ, σ) ⇒ (x - μ) / σ)
  transforms.Normalize(0.5, 0.5),
  # # １階テンソル化
  transforms.Lambda(lambda x: x.view(-1)),
])
# 検証データ
test_set = datasets.CIFAR10(
  root=data_root,
  train=False,
  download=True,
  # transform=transform
)

# image, label = test_set[0]
# print('検証データの件数:', len(test_set))
# print('検証データの型：', type(image))
# print('検証データのshape：', image.shape)
# print('最小値：', image.data.min())
# print('最大値：', image.data.max())

# 最初のROW_NUM×COL_NUM件だけ、予測・正解データ付きでイメージを表示
ROW_NUM = 5
COL_NUM = 10
fig = plt.figure(figsize=(COL_NUM, ROW_NUM + 1))
plt.suptitle('予測:正解')
for i in range(COL_NUM * ROW_NUM):
  image, label = test_set[i]
  ax = plt.subplot(ROW_NUM, COL_NUM, i + 1)
  # イメージ表示
  plt.imshow(image)
  ax.set_title(f'{label}')
  ax.get_xaxis().set_visible(False)
  ax.get_yaxis().set_visible(False)
plt.show()


cuda:0


100%|██████████| 170M/170M [00:03<00:00, 44.1MB/s]


検証データの件数: 10000
検証データの型： <class 'PIL.Image.Image'>


AttributeError: 'Image' object has no attribute 'shape'